In [ ]:
%matplotlib widget
import matplotlib

import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("./underactuated/src")

import math

from underactuated.meshcat_rigid_body_visualizer import(MeshcatRigidBodyVisualizer)
from underactuated.planar_rigid_body_visualizer import PlanarRigidBodyVisualizer

from pydrake.all import(
    DiagramBuilder,
    RigidBodyTree,
    RigidBodyFrame,
    RigidBodyPlant,
    SignalLogger,
    Simulator,
    AddModelInstanceFromUrdfFile,
    FloatingBaseType)


In [ ]:
from pydrake.all import VectorSystem

class Controller(VectorSystem):
# konstruktor klasy
        def __init__(self, value):
# wywolanie konstuktora klasy nadrzednej VectorSystem
# Podane parametry odpowiadaja opiektowi z 1 portem wyjsciowym i bez portow wejsciowych
            VectorSystem.__init__(self, 0, 1)
# ustawienie pola klasy na wartosc poczatkowa 0
# ta wartosc stanowi w tym przypadku wartosc momentu napedowego
            self.value = value
# funkcja odpowiadajaca za wyliczanie wartosci wyjsciowej output podczas symulacji
        def _DoCalcVectorOutput(self, context, unused, unused2, output):
            output[:] = [self.value]  # zawsze zwraca wartosc 0

In [ ]:
# Stworzenie obiektu DiagramBuilder, ktory bedzie zawieral elementy symulacji i opis polaczen między nimi
builder = DiagramBuilder()

# Przygotowanie elementow (blokow) do umieszczenia na diagramie
controller = Controller(0.0)   # Stworzenie obiektu kontrolera

# Stworzenie obiektu RigidBodyTree
rbt = RigidBodyTree()
# Stworzenie obiektu RigidBodyFrame do okreslenia polozenia modelu w symulacji
robot_base_frame = RigidBodyFrame("robot_base_frame", rbt.world(),[0.0, 0, 0], [0, 0, 0])
# Dodanie modelu opisanego za pomoca pliku urdf w polozeniu robot_base_frame do rbt
AddModelInstanceFromUrdfFile("wahadlo.urdf", FloatingBaseType.kFixed, robot_base_frame, rbt)
# Stworzenie obiektu RigidBodyPlant na podstawie rtb
rbplant = RigidBodyPlant(rbt)

#pbrv = MeshcatRigidBodyVisualizer(rbt, draw_timestep=0.01)
pbrv = PlanarRigidBodyVisualizer(rbt)

# Umieszczenie elementow na diagramie
controller_system = builder.AddSystem(controller)  # Dodanie prostego sterowanika do diagramu
pendulum = builder.AddSystem( rbplant ) # Dodanie modelu robota, w miejsce argumentu nalezy wisac nazwe wlasciwej zmiennej
visualizer = builder.AddSystem(pbrv)

logger = builder.AddSystem(SignalLogger(pendulum.get_output_port(0).size()))
rate = 60.0
logger._DeclarePeriodicPublish(1. / rate, 0.0)

# Stworzenie polaczen między blokami
builder.Connect(controller_system.get_output_port(0), pendulum.get_input_port(0)) # polczenie wyjscia 0 kontrolera do wejscia 0 modelu
builder.Connect(pendulum.get_output_port(0), logger.get_input_port(0))
builder.Connect(pendulum.get_output_port(0), visualizer.get_input_port(0))
# TODO: uzupelnic brakujace polaczenia

In [ ]:
# Stworzenie symulacji
diagram = builder.Build()
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
simulator.set_publish_every_time_step(False)
# Uzyskanie dostepu do wektora wartosci poczatkowych i ich ustawienie
# Uwaga: wymiar wektora to 2n gdzie n to ilosc zlaczy manipulator, w wektorze należy podacwartosci kolejno pozycji poczatkowych wszystkich zlaczy a następnie wszystkich predkoscipoczatkowych
state = simulator.get_mutable_context().get_mutable_continuous_state_vector()
state.SetFromVector([math.radians(90.0), 0.0])
# Uruchomienie symulacji o dlugosci t w sekundach
simulator.StepTo(10)

In [ ]:
nq = rbt.get_num_positions()
plt.figure()
color = iter(plt.cm.rainbow(np.linspace(0, 1, nq)))

plt.subplot(2, 1, 1)
for i in range(nq):
    colorthis = next(color)
    plt.plot(logger.sample_times(),
        logger.data()[i, :],
        color=colorthis,
        linestyle='solid',
        label="q[%d]" % i)
plt.ylabel("rad")
plt.grid(True)
plt.legend(bbox_to_anchor=(0.9, 1), loc=2, borderaxespad=0.)

color = iter(plt.cm.rainbow(np.linspace(0, 1, nq)))
plt.subplot(2, 1, 2)
for j in range(nq,2*nq,1):
    colorthis = next(color)
    plt.plot(logger.sample_times(),
        logger.data()[j, :],
        color=colorthis,
        linestyle='solid',
        label="q'[%d]" % (j-nq))
plt.ylabel("rad/s")
plt.grid(True)
plt.legend(bbox_to_anchor=(0.9, 1), loc=2, borderaxespad=0.)
plt.show()